In [1]:
import time
time_ini=time.time()

In [2]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import DateOffset
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
comex_df_dum=pd.read_csv('/home/andre301267/git/Pricing-Fertilizer/DB/temp/imp_fert_select.csv',parse_dates=['date'])
comex_df_dum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 978 entries, 0 to 977
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           978 non-null    datetime64[ns]
 1   kg             978 non-null    float64       
 2   month~1        978 non-null    bool          
 3   month~2        978 non-null    bool          
 4   month~3        978 non-null    bool          
 5   month~4        978 non-null    bool          
 6   month~5        978 non-null    bool          
 7   month~6        978 non-null    bool          
 8   month~7        978 non-null    bool          
 9   month~8        978 non-null    bool          
 10  month~9        978 non-null    bool          
 11  month~10       978 non-null    bool          
 12  month~11       978 non-null    bool          
 13  month~12       978 non-null    bool          
 14  Product~MAP    978 non-null    bool          
 15  Product~MOP    978 non-

# LOOP

In [4]:
qq=pd.DataFrame()

for rr in range(1,6):

    predict_date=comex_df_dum.date.max()-DateOffset(months=12*rr-1)
    
    train=comex_df_dum[comex_df_dum.date<predict_date]
    test=comex_df_dum[comex_df_dum.date==predict_date]
    
    train.date=train.date.astype(int)
    test.date=test.date.astype(int)
    
    # Classificação
    
    X_train=train.drop(columns=['kg','oc'])
    y_train=train.oc
    X_test=test.drop(columns=['kg','oc'])
    
    from sklearn.ensemble import RandomForestClassifier as RFC
    RFC=RFC()
    RFC.fit(X_train,y_train)
    test=test.assign(oc_pred=RFC.predict(X_test))
    
    # Regressão
    
    train=train[train.oc==1]
    
    test=test[(test.oc==1)&(test.oc_pred==1)]
    
    X_train=train.drop(columns=['kg','oc'])
    y_train=train.kg
    X_test=test.drop(columns=['kg','oc','oc_pred'])
        
    from sklearn.ensemble import RandomForestRegressor as RFR
    RFR=RFR(max_features=10, max_depth=10)
    
    
    RFR.fit(X_train,y_train)
    test=test.assign(kg_test_rfr=RFR.predict(X_test))

    
    mes_dum=test[test.columns[test.columns.str.contains('month')]]
    
    Product_dum=test[test.columns[test.columns.str.contains('Product')]]
    
    w=(ano_dum+mes_dum+Product_dum).columns
    
    outros=test[[x for x in test.columns if x not in w]]
    
    l=[ano_dum,mes_dum,Product_dum]
    
    q1=pd.DataFrame()
    for x in l:
        t=pd.from_dummies(x,sep='~')
        q1=pd.concat([q1,t],axis=1)
    
    test_pred=pd.concat([q1,outros],axis=1)
    
    test_pred[test_pred.columns[test_pred.columns.str.contains('KG')]]=\
    test_pred[test_pred.columns[test_pred.columns.str.contains('KG')]].astype(int)
    
    test_pred.date=test_pred.date.astype('datetime64[ns]')
    
    qq=pd.concat([qq,test_pred])

NameError: name 'ano_dum' is not defined

In [ ]:
qq

In [5]:
from sklearn import metrics

In [ ]:
qqq=pd.DataFrame()
for ww in qq.Product.unique():
    qqt=qq[qq.Product==ww]
    std_lr=round(metrics.mean_squared_error(qqt['kg'],qqt['kg_test_lr'])**(1/2))
    std_rfr=round(metrics.mean_squared_error(qqt['kg'],qqt['kg_test_rfr'])**(1/2))
    std_df=pd.DataFrame({'Product':[ww],'std_lr':[std_lr],'std_rfr':[std_rfr]})
    qqq=pd.concat([qqq,std_df])

print('\n*** Standard deviation (kg) ***\n')
qqq

# Prediction

### Classification

In [ ]:
date_pred=(comex_df[comex_df.date==comex_df.date.max()][['date']].drop_duplicates()+DateOffset(months=1)).assign(key=0)

Product_pred=comex_df[['Product']].drop_duplicates().assign(key=0)

pred=date_pred.merge(Product_pred).drop(columns='key')
pred['yaer']=pred.date.dt.year
pred['month']=pred.date.dt.month

comex_df_pred=pd.concat([comex_df,pred])

comex_df_pred[['month','Product']]=comex_df_pred[['month','Product']].astype('category')
comex_df_pred_dum=pd.get_dummies(comex_df_pred,prefix_sep='~')
comex_df_pred_dum.date=comex_df_pred_dum.date.astype(int)
comex_df_pred_dum.drop(columns='yaer',inplace=True)

train=comex_df_pred_dum[comex_df_pred_dum.date<comex_df_pred_dum.date.max()].assign(oc=0)
train.loc[train.kg>0,'oc']=1

pred=comex_df_pred_dum[comex_df_pred_dum.date==comex_df_pred_dum.date.max()].drop(columns='kg')

X_train=train.drop(columns=['kg','oc'])
y_train=train.oc
X_pred=pred
RFC.fit(X_train,y_train)
pred=pred.assign(oc_pred=RFC.predict(X_pred))

### Regression

In [19]:
train=train[train.oc==1]

In [20]:
pred=pred[pred.oc_pred==1]

In [21]:
X_train=train.drop(columns=['kg','oc'])

In [22]:
y_train=train.kg

In [23]:
X_pred=pred.drop(columns=['oc_pred'])

In [ ]:
RFR.fit(X_train,y_train)

In [ ]:
pred=pred.assign(kg_pred_rfr=RFR.predict(X_pred))
pred

In [ ]:
LR.fit(X_train,y_train)

In [ ]:
pred=pred.assign(kg_pred_lr=LR.predict(X_pred))
pred

In [28]:
mes_d=pd.from_dummies(pred[pred.columns[pred.columns.str.contains('MES')]],sep='~')

In [29]:
Product_d=pd.from_dummies(pred[pred.columns[pred.columns.str.contains('Product')]],sep='~')

In [30]:
o=pred[pred.columns[~((pred.columns.str.contains('MES'))+(pred.columns.str.contains('Product')))]]

In [31]:
pred=pd.concat([o,mes_d,Product_d],axis=1)

In [32]:
pred.date=pred.date.astype('datetime64[ns]')
pred=pred.assign(yaer=pred.date.dt.year)

In [33]:
pred_tab=pred.merge(qqq)

In [34]:
pred_tab[pred_tab.columns[[2,3,7,8]]]=(pred_tab[pred_tab.columns[[2,3,7,8]]]/10**6).astype(int)

In [35]:
pred_tab.rename(columns={'kg_pred_rfr':'mil_ton_rfr','kg_pred_lr':'mil_ton_lr'},inplace=True)

# Previsão

In [ ]:
pred_tab[pred_tab.columns[[6,4,5,3,7,2,8]]]\
.assign(max_features=max_features).assign(max_depth=max_depth).assign(poly_deg=poly_deg)

In [37]:
# Convert the DataFrame to markdown format
markdown_table = pred_tab.to_markdown(index=False)

# Define the path of the markdown file
markdown_file = 'report.md'

# Write the markdown table into the file
with open(markdown_file, 'w') as f:
    f.write("# Data Report\n\n")
    f.write(markdown_table)
    f.write("\n")

# Graf

In [ ]:
results=pd.concat([
    qq.drop(columns=['kg_test_rfr','kg_test_lr']).assign(tipo='real'),
    qq.drop(columns=['kg_test_rfr','kg']).rename(columns={'kg_test_lr':'kg'}).assign(tipo='test_lr'),
    qq.drop(columns=['kg','kg_test_lr']).rename(columns={'kg_test_rfr':'kg'}).assign(tipo='test_rfr'),
    pred.drop(columns=['kg_pred_lr']).rename(columns={'kg_pred_rfr':'kg'}).assign(tipo='pred_rfr'),
    pred.drop(columns=['kg_pred_rfr']).rename(columns={'kg_pred_lr':'kg'}).assign(tipo='pred_lr')
    
])

results.yaer=results.yaer.astype(int)

results.loc[results.tipo.str.contains('test'),'tipo']=results[results.tipo.str.contains('test')].tipo.str.replace('test','pred')

sns.relplot(data=results,x='yaer',y='kg',hue='tipo',col='Product',height=3,facet_kws={'sharey': False, 'sharex': True})\
.set_xticklabels(rotation=30).fig.suptitle(f"Importações - {UF[0]} - Mês {predict_date.month}",y=1.2)
plt.show()

In [ ]:
time_end=time.time()
print(f"Duração: {round((time_end-time_ini)/20,1)} minutos.")